# Parameter estimation on GW170817 temp

### Instalization

In [6]:
import bilby # Importing the bilby module for gravitational wave analysis
from gwpy.timeseries import TimeSeries # Importing the TimeSeries class from gwpy.timeseries module

logger = bilby.core.utils.logger # Initializing the logger for logging purposes
outdir = 'outdir'  # Setting the output directory path to 'outdir'
label = 'GW170817' # Setting the label for the analysis as 'GW150914'
time_of_event = bilby.gw.utils.get_event_time(label)

### Data Setup

In [7]:
trigger_time = 1187008882 # The trigger time for the analysis

roll_off = 0.2  # Roll off duration of tukey window in seconds, default is 0.4s
duration = 32  # Analysis segment duration (Duration of the analysis segment in seconds)
post_trigger_duration = 2  # Time between trigger time and end of segment
end_time = trigger_time + post_trigger_duration # Calculating the end time of the segment
start_time = end_time - duration # Calculating the start time of the segment

psd_duration = 1024   # Duration of the power spectral density estimation
psd_start_time = start_time - psd_duration  # Calculating the start time of the PSD estimation
psd_end_time = start_time # The end time of the PSD estimation is the same as the start time of the segment

### Download the data

In [8]:
# We now use gwpy to obtain analysis and psd data and create the ifo_list
ifo_list = bilby.gw.detector.InterferometerList([])

# Loop over each detector (H1 and L1)
for det in ["H1", "L1", "V1"]:
    logger.info("Downloading analysis data for ifo {}".format(det))

    # Create an empty interferometer object for the detector
    ifo = bilby.gw.detector.get_empty_interferometer(det)

    # Fetch the strain data from gwpy for the specified time interval
    data = TimeSeries.fetch_open_data(det, start_time, end_time)

    # Set the strain data of the interferometer from the gwpy TimeSeries object
    ifo.strain_data.set_from_gwpy_timeseries(data)

    logger.info("Downloading psd data for ifo {}".format(det))

    # Fetch the PSD data from gwpy for the specified time interval
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)

    # Calculate the alpha parameter for the Tukey window
    psd_alpha = 2 * roll_off / duration

    # Calculate the PSD using gwpy's psd() method
    psd = psd_data.psd(
        fftlength=duration,
        overlap=0,
        window=("tukey", psd_alpha),
        method="median"
    )

    # Create a PowerSpectralDensity object using the frequency and PSD arrays
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, psd_array=psd.value)

    # Append the interferometer object to the ifo_list
    ifo_list.append(ifo)

logger.info("Saving data plots to {}".format(outdir))

# Check if the output directory exists, if not, create it
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)

# Plot the data using the interferometer list and save the plots to the output directory
ifo_list.plot_data(outdir=outdir, label=label)


13:03 bilby INFO    : Downloading analysis data for ifo H1
13:06 bilby INFO    : Downloading psd data for ifo H1
13:10 bilby INFO    : Downloading analysis data for ifo L1
13:13 bilby INFO    : Downloading psd data for ifo L1
13:18 bilby INFO    : Downloading analysis data for ifo V1
13:21 bilby INFO    : Downloading psd data for ifo V1
13:24 bilby INFO    : Saving data plots to outdir
13:24 bilby INFO    : Generating frequency domain strain from given time domain strain.
13:24 bilby INFO    : Applying a tukey window with alpha=0.0125, roll off=0.2
13:24 bilby INFO    : Generating frequency domain strain from given time domain strain.
13:24 bilby INFO    : Applying a tukey window with alpha=0.0125, roll off=0.2
13:24 bilby INFO    : Generating frequency domain strain from given time domain strain.
13:24 bilby INFO    : Applying a tukey window with alpha=0.0125, roll off=0.2


### Create a prior

In [15]:
# CHOOSE PRIOR FILE
prior = bilby.gw.prior.BNSPriorDict(filename='GW170817.prior')
deltaT = 0.1
prior['geocent_time'] = bilby.core.prior.Uniform(
    minimum=time_of_event - deltaT / 2,
    maximum=time_of_event + deltaT / 2,
    name='geocent_time',
    latex_label='$t_c$',
    unit='$s$')

### Create a waveform generator

In [16]:
# GENERATE WAVEFORM
# OVERVIEW OF APPROXIMANTS:
# https://www.lsc-group.phys.uwm.edu/ligovirgo/cbcnote/Waveforms/Overview
duration = None  # duration and sampling frequency will be overwritten
# to match the ones in interferometers.
# All keyword arguments are passed to
# `gwpy.timeseries.TimeSeries.fetch_open_data()'
kwargs = {}
# Data are stored by LOSC at 4096 Hz, however
# there may be event-related data releases with a 16384 Hz rate.
kwargs['sample_rate'] = 4096
# For O2 events a "tag" is required to download the data.
# CLN = clean data; C02 = raw data
kwargs['tag'] = 'C02'
sampling_frequency = kwargs['sample_rate']
start_time = 0  # set the starting time of the time array
waveform_arguments = {
    'waveform_approximant': 'IMRPhenomPv2_NRTidal', 'reference_frequency': 20}

source_model = bilby.gw.source.lal_binary_neutron_star
convert_bns = bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    start_time=start_time,
    frequency_domain_source_model=source_model,
    parameter_conversion=convert_bns,
    waveform_arguments=waveform_arguments,)

13:32 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters


### Create a likelihood

In [21]:
# CHOOSE LIKELIHOOD FUNCTION
# Time marginalisation uses FFT.
# Distance marginalisation uses a look up table calculated at run time.
# Phase marginalisation is done analytically using a Bessel function.
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list, waveform_generator, priors=prior, time_marginalization=True,
    phase_marginalization=True, distance_marginalization=True)

13:34 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
13:34 bilby INFO    : Building lookup table for distance marginalisation.


  0%|          | 0/400 [00:00<?, ?it/s]

In [22]:
# RUN SAMPLER
# Implemented Samplers:
# LIST OF AVAILABLE SAMPLERS: Run -> bilby.sampler.implemented_samplers
# conversion function = bilby.gw.conversion.generate_all_bns_parameters
npoints = 512
sampler = 'dynesty'
result = bilby.run_sampler(
    likelihood,
    prior,
    outdir=outdir,
    label=label,
    sampler=sampler,
    npoints=npoints,
    use_ratio=False,
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters)

result.plot_corner()

13:35 bilby INFO    : Running for label 'GW170817', output will be saved to 'outdir'
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
13:35

1it [00:00, ?it/s]

13:42 bilby INFO    : Run interrupted by signal 2: checkpoint and exit on 130
13:42 bilby INFO    : Written checkpoint file outdir/GW170817_resume.pickle


Exception while calling loglikelihood function:
  params: [ 1.19726505e+00  5.54340068e-01  3.79467504e-02  9.24840037e-03
  7.94315868e-01  9.73075884e-01  5.34517080e+00  4.72839514e+00
 -1.67571786e-01  3.71309033e+00  1.88714615e+00  1.77436164e+00
  5.63637883e+02  3.22162332e+01 -1.52187314e-04]
  args: []
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/dynesty/dynesty.py", line 910, in __call__
    return self.func(np.asarray(x).copy(), *self.args, **self.kwargs)
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/core/sampler/dynesty.py", line 55, in _log_likelihood_wrapper
    return _sampling_convenience_dump.likelihood.log_likelihood()
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 853, in log_likelihood
    return self.log_likelihood_ratio() + self.noise_log_likelihood()
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 412, in log_likelihood_ratio
    log_l = self.compute_log_likelihood_from_snrs(total_snrs)
  File "/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/bilby/gw/likelihood/base.py", line 432, in compute_log_likelihood_from_snrs
    log_l = se

SystemExit: 130

/Users/dhruv/miniconda3/envs/igwn-py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
